# Setup

In [1]:
import sys
from pathlib import Path

%load_ext autoreload
%autoreload 2

root_path = Path("/home/olivieri/exp").resolve()
src_path = root_path / "src"
sys.path.append(f"{str(src_path)}")

In [ ]:
import pandas as pd
from glob import glob
import os

from prompter import *
from data import *

In [3]:
# Setting
BY_MODEL = "LRASPP_MobileNet_V3"
IMAGE_RESIZING_MODE = "letterboxed"
OUTPUT_MODE = "points"

In [4]:
root_exp_path = get_selected_annots_path(BY_MODEL, IMAGE_RESIZING_MODE, OUTPUT_MODE) / "eval_prs" / "test_color_maps"

exps_path = glob(f"{root_exp_path}/*")
exps_names = [os.path.splitext(os.path.basename(path))[0] for path in exps_path]
exps_names

['cmap_as_img', 'cmap_as_names', 'cmap_as_patches']

# Prompt Assessment

In [5]:
df_mean_pred_prs = pd.DataFrame()
df_mean_score_prs = pd.DataFrame() 

for exp_path, exp_name in zip(exps_path, exps_names):
    
    variation_paths = glob(f"{exp_path}/*.jsonl")
    variations_names = [os.path.splitext(os.path.basename(path))[0] for path in variation_paths]

    df_pred_prs = pd.DataFrame()
    df_score_prs = pd.DataFrame() 

    for variation_name in variations_names:

        # TODO: check if data points they have matching states
        
        eval_prs = get_many_eval_pr(BY_MODEL, IMAGE_RESIZING_MODE, OUTPUT_MODE, f"test_color_maps/{exp_name}", variation_name, return_state=False)

        _df_all_prs = pd.DataFrame.from_dict(eval_prs, orient='index')
        _df_pred_prs = _df_all_prs["pred"].map(lambda x: x if x is None else x == "correct")
        _df_score_prs = _df_all_prs["score"]
        _df_reason_prs = _df_all_prs["reason"]

        df_pred_prs = pd.concat([df_pred_prs, _df_pred_prs], axis=1)
        df_score_prs = pd.concat([df_score_prs, _df_score_prs], axis=1)
    
    df_pred_prs.columns = variations_names
    df_score_prs.columns = variations_names

    df_mean_pred_prs = pd.concat([df_mean_pred_prs, df_pred_prs.mean(axis=0)], axis=1)
    df_mean_score_prs = pd.concat([df_mean_score_prs, df_score_prs.mean(axis=0)], axis=1)

df_mean_pred_prs.columns = exps_names
df_mean_score_prs.columns = exps_names


## Pred

In [6]:
df_mean_pred_prs

,cmap_as_img,cmap_as_names,cmap_as_patches
ConcatMasks_Sc_Hz,0.80,0.60,0.60
ConcatMasks_Sc_Vr,0.60,0.65,0.80
ConcatMasks_Ovr_Hz,0.50,0.45,0.75
ConcatMasks_Ovr_Vr,0.35,0.40,0.75
ConcatMasks_NoSc_Hz,0.50,0.40,0.30
ConcatMasks_NoSc_Vr,0.40,0.35,0.50
SepMasks_NoSc,0.65,0.60,0.65
SepMasks_Ovr,0.65,0.75,0.65
SepMasks_Sc,0.70,0.55,0.70
ArrayMasks_Imgs,0.75,0.65,0.50


In [7]:
df_mean_pred_prs.describe().iloc[[1, 2, 3, 7]]

,cmap_as_img,cmap_as_names,cmap_as_patches
mean,0.590909,0.550000,0.627273
std,0.141100,0.130384,0.145540
min,0.350000,0.350000,0.300000
max,0.800000,0.750000,0.800000


## Score

In [8]:
df_mean_score_prs

,cmap_as_img,cmap_as_names,cmap_as_patches
ConcatMasks_Sc_Hz,3.70,3.30,3.30
ConcatMasks_Sc_Vr,3.20,3.45,3.75
ConcatMasks_Ovr_Hz,2.85,2.65,3.60
ConcatMasks_Ovr_Vr,2.50,2.45,3.60
ConcatMasks_NoSc_Hz,2.75,2.10,2.35
ConcatMasks_NoSc_Vr,2.25,2.15,2.60
SepMasks_NoSc,3.40,3.05,3.25
SepMasks_Ovr,3.45,3.60,3.45
SepMasks_Sc,3.35,3.10,3.45
ArrayMasks_Imgs,3.60,3.25,3.10


In [9]:
df_mean_score_prs.describe().iloc[[1, 2, 3, 7]]

,cmap_as_img,cmap_as_names,cmap_as_patches
mean,3.118182,2.954545,3.268182
std,0.467051,0.531251,0.434898
min,2.250000,2.100000,2.350000
max,3.700000,3.600000,3.750000
